In [1]:
from randomgen import RandomNetworkGenerator
import json
import requests
import time

In [2]:
schema = json.load(open("../../../metadata/relations.json", "r"))
generator = RandomNetworkGenerator(schema)

custom_counts = {
    "BusinessUnit": 1,
    "ProductFamily": 2,
    "ProductOffering": 3,
    "Facility": 2,
    "Parts": 4,
    "Warehouse": 5,
    "Supplier": 5,
}

In [3]:
# API_URL = "https://bug-hardy-obviously.ngrok-free.app"
API_URL = "http://localhost:8000"

version = "dynamic_small_10_bulk_test_v3"

In [ ]:
create_ops = generator.create_network(custom_counts)

print(f"Generated {len(create_ops)} operations")

print(create_ops)



In [ ]:
bulk_create_payload = {
    "version": version,
    "action": "bulk_create",
    "type": "schema",
    "timestamp": create_ops[0]["timestamp"],
    "payload": []
}

for op in create_ops:
    bulk_create_payload["payload"].append(op["payload"])

print(bulk_create_payload)

requests.post(f"{API_URL}/schema/live/update", json=bulk_create_payload)

In [ ]:
update_cycles = 2
for _ in range(update_cycles):
    update_ops = generator.generate_updates(
        node_updates=10,
        edge_updates=10
    )
    print(f"Generated {len(update_ops)} update operations")

    bulk_update_payload = {
        "version": version,
        "action": "bulk_update",
        "type": "schema",
        "timestamp": update_ops[0]["timestamp"],
        "payload": []
    }

    for op in update_ops:
        bulk_update_payload["payload"].append(op["payload"])

    print(bulk_update_payload)

    requests.post(f"{API_URL}/schema/live/update", json=bulk_update_payload)
    time.sleep(1)

In [ ]:
delete_cycles = 2
for _ in range(delete_cycles):
    delete_ops = generator.generate_deletions(
        node_deletions=5,
        edge_deletions=5
    )
    print(f"Generated {len(delete_ops)} delete operations")

    updated_delete_ops = []

    for op in delete_ops:
        op["version"] = version
        updated_delete_ops.append(op)

    requests.post(f"{API_URL}/schema/live/update/bulk", json=updated_delete_ops)